# Coffee Farm Condition Monitoring

<img src="images/farm_landscape.jpeg" alt="farm" width="600"/>

This notebook provides code to analyze data collected using sensors deployed at the Dedan Kimathi University of Technology Coffee farm and display it for visual inspection.

We have deployed the following sensors
1. Ambient Temperature
1. Relative Humidity
1. Soil Temperature 
1. Soil Moisture

The aim is to use these data to help plan farming activities such as irrigation and fungicide application.

The sensors are connected to a [Nucleo F446re](https://os.mbed.com/platforms/ST-Nucleo-F446RE/) development board running code contained in this [repo](https://github.com/ciiram/nyeri-coffee).

## Requirements

1. [pandas](https://pandas.pydata.org/)
1. [matplotlib](https://matplotlib.org/)
1. [GPy](https://sheffieldml.github.io/GPy/)

In [ ]:
%matplotlib inline
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from datetime import datetime, timedelta

## Plots of Ambient Temperature in The Initial Deployment

We will examine plots of ambient temperature recorded by three devices in the farm in September 2018.

In [ ]:
#load the data
df = pd.read_csv('data/2018-09-30-Data.csv')

# convert data type of time to datetime
df[['_source.time']] = df[['_source.time']].apply(pd.to_datetime)

# we are interested in time, ambient temperature and humidity, and soil temperature and humidity
df_coffee_farm = df[['_source.time','_source.dev_id','_source.temperature_2', '_source.relative_humidity_3', '_source.analog_in_4', '_source.analog_in_5']]
df_coffee_farm.columns = ['time','dev_id','Ambient Temperature', 'Relative Humidity', 'Soil Temperature', 'Soil Moisture']

# sorting based on time
df_coffee_farm.sort_values(by='time');


In [ ]:
#plots for various devices
for dev_name, gdf in df_coffee_farm.groupby('dev_id'):
        plt.figure()
        plt.title(dev_name)
        plt.plot(gdf.time + timedelta(hours=3), gdf['Ambient Temperature'],'bo') # correction for time zone
        plt.legend(['Ambient Temperature', 'Soil Temperature']);
        plt.xticks(rotation=45);
        plt.ylim([0, 50]);
        plt.xlabel('Time');
        plt.ylabel('Temperature (Celcius)');
        plt.tight_layout()
        plt.savefig('../figures/' + dev_name + '.jpg', dpi=300)

## Second Deployment

In a second deployment, we monitored ambient and soil temperature as well as soil moisture. Our deployment coincided with the onset of the rainy season and a jump in the moiture level is noticed. Also, on the 9th of October the battery failed and this can be seen in the data from the soil temperature and moisture sensors is unreliable. Also, the soil temperature sensor appears noisy.

In [ ]:
#load the data
ambient_temperature_df = pd.read_csv('data/ambient_temperature.csv', sep=";")
soil_temperature_df = pd.read_csv('data/soil_temperature.csv', sep=";")
soil_moisture_df = pd.read_csv('data/soil_moisture.csv', sep=";")

# convert data type of time to datetime and value to numeric
data_frames = [ambient_temperature_df, soil_temperature_df, soil_moisture_df]

for df in data_frames:
    df[['Time']] = df[['Time']].apply(pd.to_datetime)
    df[['Value']] = pd.to_numeric(df['Value'], errors='coerce')

plt.figure()
plt.plot(ambient_temperature_df[['Time']], ambient_temperature_df[['Value']], 'bo');
plt.xticks(rotation=45);
plt.ylim([5, 30]);
plt.xlabel('Time');
plt.ylabel('Ambient Temperature (Celcius)');

plt.figure()
plt.plot(soil_temperature_df[['Time']], soil_temperature_df[['Value']], 'go');
plt.xticks(rotation=45);
plt.ylim([5, 30]);
plt.xlabel('Time');
plt.ylabel('Soil Temperature (Celcius)');

plt.figure()
plt.plot(soil_moisture_df[['Time']], soil_moisture_df[['Value']], 'bo');
plt.xticks(rotation=45);
plt.ylim([-5, 100]);
plt.xlabel('Time');
plt.ylabel('Soil Moisture (%)');


## Modelling

We will experiment with curve fitting using the data we have collected. We will fit Gaussian process models with various kernels and assess the fits. We will work with the temperature data from *aws-dev-02*.

In [ ]:
import GPy
import numpy as np
from IPython.display import display

#prepare the data
dev_groups = df_coffee_farm.groupby('dev_id')

# get data corresponding to device 2
device_2 = [grp for name, grp in dev_groups][1] 
device_2 = device_2.sort_values(by='time');

#get the temperature and time
temperature = device_2.values[:,2]
time_day = [value.timestamp() for value in device_2.values[:,0]] # get the timestamps
time_day = np.array(time_day)

#normalize to days
time_day = time_day - min(time_day)
time_day /= (24 * 60 * 60)

# get the first 7 days
temperature = temperature[time_day < 7]
time_day = time_day[time_day < 7]


We visualise the data

In [ ]:
plt.figure()
plt.plot(time_day, temperature, 'bo')
plt.xlim([-.5,7])
plt.ylim([0, 30])
plt.xlabel('Time (days)');
plt.ylabel('Temperature (Celcius)');

We fit a Gaussian process with a radial basis function kernel. (If the optimization fails, simply restart it.)

In [ ]:
#RBF Kernel
kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
m = GPy.models.GPRegression(time_day[:, None],temperature[:, None],kernel)
m.optimize_restarts(num_restarts = 10)
display(m)

Let's plot the fit

In [ ]:
plt.figure()
m.plot();
plt.ylim([-30, 50]);
plt.xlabel('Time (days)');
plt.ylabel('Temperature (Celcius)');

The fit with the RBF kernel does not do a good job of filling in the missing data. In the region without data, the posterior fit is similar to the prior which results in the interpolant passing through zero. It is never that cold in Nyeri!

We now try a periodic kernel on the same data which takes into account the observation that temperature variations are periodic. (If the optimization fails, simply restart it.)

In [ ]:
periodic_kernel = GPy.kern.PeriodicMatern32(input_dim=1, variance=1., lengthscale=1., period=1.)
m = GPy.models.GPRegression(time_day[:, None],temperature[:, None], periodic_kernel)
m.optimize_restarts(num_restarts = 10)

In [ ]:
plt.figure()
m.plot();
plt.ylim([-30, 50]);
plt.xlabel('Time (days)');
plt.ylabel('Temperature (Celcius)');

The fit is still not good. The prior with a zero mean does not fit the data.
We need to add bias. (If the optimization fails, simply restart it.)



In [ ]:
kernel_bias = GPy.kern.Bias(1)

m = GPy.models.GPRegression(time_day[:, None],temperature[:, None], periodic_kernel + kernel_bias)
m.optimize_restarts(num_restarts = 20)

# plot the fit
plt.figure()
m.plot();
plt.ylim([-30, 50]);
plt.xlabel('Time (days)');
plt.ylabel('Temperature (Celcius)');

This is a much better fit and the predicted values in the region with missing data is quite reasonable.
